In [46]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [99]:
# read from melodyData.txt
with open('out/abc.txt', 'r') as f:
    parts = eval(f.read())[:5]

# one-hot encoding

encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


In [118]:
sequence_length = 10

X = []
Y = []
# given data_encoded, generate training data by looping 
for i in range(len(data_encoded)):
    for j in range(len(data_encoded[i]) - sequence_length):
        X.append(data_encoded[i][j:j + sequence_length])
        Y.append(data_encoded[i][j + sequence_length])

X_onehot = []
for seq in X:
    onehot = np.zeros((len(seq), len(encodings)))
    for note_index in range(len(seq)):
        onehot[note_index][seq[note_index]] = 1
    X_onehot.append(onehot)
X = X_onehot


# split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

print (np.array(X_train).shape)
print (np.array(Y_train).shape)
print (np.array(X_test).shape)
print (np.array(Y_test).shape)

(20, 10, 4)
(20,)
(6, 10, 4)
(6,)
[3, 2, 3, 2, 3, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 1, 3, 1, 2, 2]


In [120]:
#split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
# T    tf.keras.layers.Input(shape=(sequence_length, 1)), X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(sequence_length, len(encodings))),
    #tf.keras.layers.Input(shape=(sequence_length, 1)),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.LSTM(128, return_sequences=True),
    # tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='softplus'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(Y_train), epochs=20)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.7808
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4500 - loss: 1.3696
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4500 - loss: 1.1158
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4500 - loss: 0.9596
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9000 - loss: 0.8591
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9000 - loss: 0.7999
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9000 - loss: 0.7582
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9000 - loss: 0.7224
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9000 - loss: 0.6815
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9000 - loss: 0.6348
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9000 - loss: 0.5846
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9000 - loss: 0.5343

In [121]:
loss, accuracy = model.evaluate(np.array(X_test), np.array(Y_test), verbose=1)
print(f'Loss: {loss}, Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 1.0000 - loss: 0.5697
Loss: 0.5696602463722229, Accuracy: 1.0


In [135]:
import copy
# given a sequence, generate the next note (up to ten times)
seq = copy.deepcopy(X_train[4])
total_seq = seq[:]
for i in range(10):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    print (pred)
    # append to sequence the one-hot encoding of pred
    one_hot = np.zeros((len(encodings)))
    one_hot[pred] = 1
    total_seq = np.append(total_seq, [one_hot], axis=0)
    # set sequence to be the last ten values of total_seq
    seq = total_seq[-10:]
    #print (seq)

    #total_seq.append(pred)
    # total seq is a numpy array, can't just append things to it

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}
    
"""
for note in total_seq:
    pitch, dur = decodings[np.argmax(note)]
    print (mapping[pitch % 12], dur)
"""
# convert numbers to musical note letters
# 60 = C

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3


'\nfor note in total_seq:\n    pitch, dur = decodings[np.argmax(note)]\n    print (mapping[pitch % 12], dur)\n'